In [1]:
!conda install ffmpeg -y

Channels:
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



In [2]:
!pip install fastapi uvicorn python-multipart moviepy openai-whisper

In [ ]:
# coding: utf-8
# (선택) Jupyter에서 실행 중일 때만 의존성 설치
try:
    get_ipython  # noqa
    get_ipython().system('conda install ffmpeg -y')
    get_ipython().system('pip install fastapi uvicorn python-multipart moviepy openai-whisper')
except NameError:
    pass
from fastapi import FastAPI, File, UploadFile, Form
from moviepy import VideoFileClip
import whisper
import torch
import tempfile
import nest_asyncio
import uvicorn
import traceback
import subprocess   # 이전 오류 보완을 위해 유지
import os
import requests
import time
import re
app = FastAPI()
# Whisper 모델 로딩
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("large", device=device)
@app.post("/process_video")
async def process_video(file: UploadFile = File(...), text: str = Form(...)):
    print("process_video 시작")
    try:
        # 업로드 파일명/확장자 확인
        filename = (file.filename or "").lower()
        if not filename.endswith((".mp4", ".webm")):
            return {"status": "error", "message": "mp4 또는 webm 파일만 지원합니다."}
        ext = ".webm" if filename.endswith(".webm") else ".mp4"
        # 업로드 파일을 메모리에서 읽기
        video_bytes = await file.read()
        # 임시로 업로드 확장자에 맞춰 저장
        with tempfile.NamedTemporaryFile(suffix=ext, delete=False) as temp_video:
            temp_video.write(video_bytes)
            video_path = temp_video.name
        audio_path = None
        # 1) MoviePy로 오디오 추출 시도
        try:
            with VideoFileClip(video_path) as video_clip:
                audio_clip = video_clip.audio
                if audio_clip is None:
                    # 오디오 트랙 없음
                    if os.path.exists(video_path):
                        os.unlink(video_path)
                    return {"status": "error", "message": "동영상에서 음성을 찾을 수 없습니다."}
                # Whisper 친화적 WAV(PCM s16le, Mono, 16kHz)로 저장
                with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
                    audio_path = temp_audio.name
                audio_clip.write_audiofile(
                    audio_path,
                    codec="pcm_s16le",
                    fps=16000,
                    ffmpeg_params=["-ac", "1"],  # mono
                    verbose=False,
                    logger=None,
                )
                audio_clip.close()
        except Exception as moviepy_err:
            # 2) MoviePy 실패 시 ffmpeg CLI로 안전 추출 (webm 코덱 호환성 대비)
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_audio:
                audio_path = temp_audio.name
            cmd = [
                "ffmpeg", "-y",
                "-i", video_path,
                "-vn",                    # 비디오 제거
                "-acodec", "pcm_s16le",  # PCM s16
                "-ar", "16000",          # 16kHz
                "-ac", "1",              # mono
                audio_path,
            ]
            try:
                subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            except subprocess.CalledProcessError as ffmpeg_err:
                # 임시 파일 정리 후 에러 반환
                if os.path.exists(video_path):
                    os.unlink(video_path)
                if os.path.exists(audio_path):
                    os.unlink(audio_path)
                return {
                    "status": "error",
                    "message": f"오디오 추출 실패 (MoviePy/ffmpeg): {moviepy_err} / {ffmpeg_err}",
                }
        # 3) Whisper로 음성 → 텍스트 변환
        result = model.transcribe(audio_path, fp16=(device == "cuda"))
        transcribed_text = (result.get("text") or "").strip()
        # 임시 파일 정리
        if os.path.exists(video_path):
            os.unlink(video_path)
        if audio_path and os.path.exists(audio_path):
            os.unlink(audio_path)
        
        def translate_text(text, source_lang='en', target_lang='ko'):
            base_url = 'https://lingva.ml/api/v1'
            url = f"{base_url}/{source_lang}/{target_lang}/{requests.utils.quote(text)}"
            response = requests.get(url)
            if response.status_code == 200:
                result = response.json()
                return result.get('translation', '')
            else:
                print(f"번역 API 호출 실패: {response.status_code}")
                return None
        print("sst된 음성 : ", transcribed_text)
        print("답변 생성중 ~~~")
        
        API_URL = "http://localhost:4891/v1/chat/completions"
        headers = {"Content-Type": "application/json"}
        data = {
          "model": "DeepSeek-R1-Distill-Qwen-14B",
          #"model": "DeepSeek-R1-Distill-Qwen-7B",
          #"model": "DeepSeek-R1-Distill-Qwen-1.5B",
          "messages": [
            {
              "role": "user",
              "content": f"당신은 전문 면접코치입니다. [{text}] 라는 질문에 대해 면접자가 [{transcribed_text}] 이라고 답했을 때에 아래 형식을 항상 준수하여 답변하세요. [답변 형식] [1. 답변의 긍정적 측면 : ~~~ 2. 답변의 개선할 점 : ~~~ 3. 개선된 면접자의 답변 : ~~~] 위와 같은답변 형식으로 총 300자 내외로 한국어로 번역하지 말고 영어로 답변하세요."
            }
          ],
          "max_tokens": 2048,
          "temperature": 0.7
        }
        
        start_time = time.time()
        
        response = requests.post(API_URL, headers=headers, json=data)
        
        end_time = time.time()
        elapsed = end_time - start_time
        
        try:
            if response.status_code == 200 and response.text:
                result = response.json()
                answer = result['choices'][0]['message']['content']
                # <think> 태그 제거
                clean_text = re.sub(r'<think>.*?</think>', '', answer, flags=re.DOTALL).strip()
                print("GPT4All 답변 (영어):")
                print(clean_text)
        
                print("\n번역 중...")
                translated_answer = translate_text(clean_text, source_lang='en', target_lang='ko')
                if translated_answer:
                    print("\n번역된 답변 (한국어):")
                    print(translated_answer)
                else:
                    print("번역 실패")
                    
                print(f"\n질문-답변에 걸린 시간: {elapsed:.2f}초")
            else:
                print('API 호출 실패:', response.status_code, response.text)
        except Exception as e:
            print('예외 발생:', e)
            print('서버 응답 원문:', response.text)
        # 결과 반환
        return "질문 : " + text + "stt : " +transcribed_text + "응답 : " + translated_answer
    except Exception as e:
        error_traceback = traceback.format_exc()
        # 예외 발생 시 임시 파일 정리
        try:
            if 'video_path' in locals() and os.path.exists(video_path):
                os.unlink(video_path)
            if 'audio_path' in locals() and audio_path and os.path.exists(audio_path):
                os.unlink(audio_path)
        finally:
            return {
                "status": "error",
                "message": str(e),
                "traceback": error_traceback
            }
# Jupyter notebook에서 서버 실행
try:
    nest_asyncio.apply()  # Jupyter 환경용
    uvicorn.run(app, host="0.0.0.0", port=8000, http='h11')
except RuntimeError:
    # 이미 실행 중인 이벤트 루프가 있으면 무시
    pass

Channels:
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



INFO:     Started server process [21120]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


process_video 시작
sst된 음성 :  저의 강점은 다양한 사람과 원활하게 소통하는 커뮤니케이션 능력입니다. 동아리플렉스에서 팀 간 협업을 조율하며 웬만한 결과를 이끌어낸 경험이 있습니다. 반면 약점은 디테일에 지나치게 집중하는 경향입니다. 이를 보완하기 위해 우선순위 설정과 일정 단위의 루틴 실천을 하고 있습니다.
답변 생성중 ~~~
GPT4All 답변 (영어):
**Answer:**

Certainly! Here is the step-by-step explanation based on your thought process:

1. **Strengths Identification**: 
   - The candidate highlights effective communication skills as their primary strength, emphasizing their ability to work well with diverse individuals. This is a valuable trait in most professional environments.

2. **Example Provided**:
   - They provide an example from their school's Harry Potter club where they led team collaborations and achieved successful results. This demonstrates practical leadership experience.

3. **Weakness Acknowledgment**:
   - The candidate identifies being overly detail-oriented as a weakness, which can be both beneficial and detrimental depending on the role. It is particularly useful in roles requiring precision but ma